In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.common.exceptions import TimeoutException
import os
import numpy as np

In [2]:
# Get the Wuzzuf sitemap URL
sitemap_url = "https://wuzzuf.net/sitemap-job-1.xml"
response = requests.get(sitemap_url)

# Parse the XML
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "xml")
    urls = [loc.text for loc in soup.find_all("loc")]
        
else:
    print("Failed to fetch the sitemap.")

In [3]:
len(urls)

8595

In [4]:
title = []
working_hours = []
working_place = []
company = []
location = []
post_date = []
num_of_applicants = []
num_of_positions = []
experience = []
career_level = []
education = []
salary = []
job_category = []
skills = []
job_description = []
job_requirements = []

In [5]:
driver = webdriver.Firefox()

In [6]:
def get_element_text(by, value, index=0):
    try:
        elements = driver.find_elements(by, value)
        return elements[index].text.strip() if elements else "N/A"
    except:
        return "N/A"

In [7]:
data = []
for url in urls[4]:
    driver.get(url)
    sleep(2) 
    
    job_data = {
        "Title": get_element_text(By.XPATH, "//section/div/h1"),
        "Company": get_element_text(By.XPATH, "//section/div/div/div/a/span"),
        "Working Place": get_element_text(By.XPATH, "//section/div/div/a/span"),
        "Location": get_element_text(By.XPATH, "//strong/div/a"),
        "Post Date": get_element_text(By.XPATH, "//section/div/span"),
        "Number of Positions": get_element_text(By.XPATH, "//section/div/div/div/strong"),
        "Applicants": get_element_text(By.XPATH, "//div/div/span/span", index=1),
        "Career Level": get_element_text(By.XPATH, "//main//section/div/span/span"),
        "Education": get_element_text(By.XPATH, "//main//section/div/span/span", index=1),
        "Salary": get_element_text(By.XPATH, "//main//section/div/span/span", index=2),
        "Job Category": get_element_text(By.XPATH, "//main//section/div/span/span", index=3),
        "Skills": get_element_text(By.XPATH, "//div/ul/li/a/span"),
        "Job Description": " ".join([i.text for i in driver.find_elements(By.XPATH, "//div/a/span/span/span")]),
    }
    
    data.append(job_data)

In [8]:
job_df = pd.DataFrame(data)
job_df

,Title,Company,Working Place,Location,Post Date,Number of Positions,Applicants,Career Level,Education,Salary,Job Category,Skills,Job Description
0,School Head of Admissions,Full Time,On-site,,Posted 24 days ago,44,1 open position,3 To 10 Years,Experienced (Non-Manager),Not Specified,Confidential,Marketing/PR/Advertising,Marketing School Education
1,Senior Backend Developer,Full Time,Hybrid,,Posted 27 days ago,12,1 open position,More Than 5 Years,Experienced (Non-Manager),Bachelor'S Degree,Confidential,IT/Software Development,Software Development Ruby Ruby on Rails Infras...
2,Civil Engineer,Full Time,On-site,,Posted 20 days ago,72,2 open positions,3 To 5 Years,Experienced (Non-Manager),Bachelor'S Degree,8000 To 12000 EGP Per Month,Engineering - Construction/Civil/Architecture,AutoCAD Civil Engineering Design Communication...
3,Senior Tendering Engineer,Full Time,On-site,,Posted 27 days ago,48,4 open positions,5 To 8 Years,Experienced (Non-Manager),Bachelor'S Degree,Confidential,Engineering - Construction/Civil/Architecture,Civil tender Engineering Tendering


In [ ]:
job_df.to_csv("wuzzuf_jobs.csv", index=False)

In [ ]:
driver.quit()